<a href="https://colab.research.google.com/github/CodeWithAlvin/handson-ml2_practice/blob/main/Ch2_practiceQues01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# try SVR on california housing dataset

In [2]:
link="https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"

In [3]:
import urllib.request

In [4]:
data=urllib.request.urlretrieve(link,"/content/data.csv")

In [5]:
import pandas as pd
import numpy as np

In [6]:
dataset=pd.read_csv("/content/data.csv")

In [7]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [8]:
X=dataset.drop("median_house_value",axis=1)
y=dataset["median_house_value"]
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
def Processor(X):
  """
  seprating objects and numeric data and then it will be pass to pipeline and concatenated again
  """
  column=X.columns
  numbers=pd.DataFrame()
  others=pd.DataFrame()

  # SPLITTING UP D/F DATATYPES
  for i in column:
    if (X[i].dtype==object) or (X[i].dtype==str):
      others[i]=X[i]
    else:
      numbers[i]=X[i]

  #PIPELINE FOR NUMBERS
  num_pipe=Pipeline([('imputer',SimpleImputer(strategy="median")),
                     ('std_scaler', StandardScaler())
                     ])
  
  #OUTPUT FROM NUMBER PIPELINE
  res_num=num_pipe.fit_transform(numbers)

  # PIPELINE FOR OBJ OR STR
  if len(others) > 1:
    object_pipe=Pipeline([("cat", OneHotEncoder()),
                        ])
    
    # OUTPUT FROM OBJ PIPELINE
    res_obj=object_pipe.fit_transform(others).toarray()
    
    #CONCATINATING FINAL DATA 

    final_data=np.c_[res_num,res_obj]
  else  :
    final_data=res_num
    
  return final_data

In [10]:
processed_train_X=Processor(train_X)

In [11]:
model = SVR()

In [12]:
C_range = 10. ** np.arange(-3, 8)
gamma_range = 10. ** np.arange(-5, 4)
params=dict(gamma=gamma_range, C=C_range,kernel=["linear","rbf","poly"])

In [13]:
random_search=RandomizedSearchCV(model,return_train_score=True,n_iter=20,param_distributions=params)

In [14]:
#random_search.fit(processed_train_X,train_y)

In [15]:
#random_search.best_estimator_

In [23]:
Svm = SVR(C=1,gamma="auto",kernel="poly")

In [24]:
Svm.fit(processed_train_X,train_y)

SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [25]:
y_pred=Svm.predict(processed_train_X)

# evaluating the model

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [27]:
mse=mean_squared_error(train_y,y_pred)
print("RMSE : ",np.sqrt(mse))

RMSE :  118520.3897888169
